In [10]:
import azureml
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.45.0


In [8]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="d6233897-5c9f-47f9-8507-6d4ada2d5183")
resource_group = os.getenv("RESOURCE_GROUP", default="RG_Jaganlal")
workspace_name = os.getenv("WORKSPACE_NAME", default="beanleaf_classifier")
workspace_region = os.getenv("WORKSPACE_REGION", default="centralus")
cluster_name = os.getenv("CLUSTER_NAME", default="beanleaf-classifier-cluster")
dataset_path_in_datastore = os.getenv("BLOB_STORAGE_PATH", default="beanleaf")
data_local_directory = os.getenv("DATA_LOCAL_DIRECTORY", default="../0_data/beanleaf_dataset/")
experiment_name = os.getenv("EXPERIMENT_NAME", default="beanleaf_classifier_experiment")

<h3>Access your workspace</h3>

In [3]:
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
get_workspace error using subscription_id=d6233897-5c9f-47f9-8507-6d4ada2d5183, resource_group_name=RG_Jaganlal, workspace_name=beanleaf_classifier


Workspace not accessible. Change your parameters or create a new workspace below


<h3>Create a new workspace</h3>

In [4]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

Deploying KeyVault with name beanleafkeyvault8b059324.
Deploying AppInsights with name beanleafinsightsccd4d1e4.
Deployed AppInsights with name beanleafinsightsccd4d1e4. Took 14.68 seconds.
Deploying StorageAccount with name beanleafstorage78ca4e858.
Deployed StorageAccount with name beanleafstorage78ca4e858. Took 33.62 seconds.
Deployed KeyVault with name beanleafkeyvault8b059324. Took 29.91 seconds.
Deploying Workspace with name beanleaf_classifier.
Deployed Workspace with name beanleaf_classifier. Took 47.11 seconds.


<h3>Create compute resources for your training experiments</h3>

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace = ws, name = cluster_name)
    print("Found existing "+cluster_name)
except ComputeTargetException:
    print("Creating new "+cluster_name)
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D4A_V4",
                                                           min_nodes=0,
                                                           max_nodes=2)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Creating new beanleaf-classifier-cluster
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
ws = Workspace.get(name=workspace_name,
                    subscription_id=subscription_id,
                    resource_group=resource_group)
# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print("Compute Target:", compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:", ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name=experiment_name)
print("Experiment:", exp.name)

Compute Target: beanleaf-classifier-cluster
Datastore: workspaceblobstore
Experiment: beanleaf_classifier_experiment


In [ ]:
# Upload data to AzureML Datastore
ds = ws.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=data_local_directory,
            target=DataPath(ds, 'beanleaf_dataset'),
            show_progress=True, overwrite=True)

# Register file dataset with AzureML
ds = ds.register(workspace=ws, name="beanleaf", description="Bean leaves with train, test and validation folders", create_new_version=True)
print(f'Dataset {ds.name} registered.')